# Create a truth database

This notebook shows how to cull needed information from the opsim and cached star databases for the relevant truth information for the stars in the Twinkles field. Once the data is gathered we also calculate the expected flux for the objects in each visit in the Twinkles run. This is used to gather values to compare against the DM processed data in our pserv databases.

"True" in this case refers to the values that come from our LSST CATSIM database. This CATSIM database stores the LSST simulated universe model that we use to provide inputs to LSST simulations. It is important to note that this means that "true" does not refer to actual stars in sky, but to the known inputs to our simulations. More information on the LSST Simulations can be found [here](bit.ly/lsst-sims-doc). 

** Note: Requires an [Opsim database](https://www.lsst.org/scientists/simulations/opsim/opsim-v335-benchmark-surveys). Twinkles 1.1 uses kraken_1042 while Twinkles 1.3 uses minion_1016. **

In [1]:
import desc.monitor
import pandas as pd
%load_ext autoreload
%autoreload 2

/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sims_catalogs/2.3.3.3.sims+5/python/lsst/sims/catalogs/db/dbConnection.py:470: UserWarning: Duplicate object type id 25 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sims_catalogs/2.3.3.3.sims+5/python/lsst/sims/catalogs/db/dbConnection.py:470: UserWarning: Duplicate object type id 40 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want globally unique id values')
/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sims_catalogs/2.3.3.3.sims+5/python/lsst/sims/catalogs/db/dbConnection.py:470: UserWarning: Duplicate object type id 4 specified: 
Output object ids may not be unique.
This may not be a problem if you do not want globally unique id values
  'want glo

## Get cached CatSim stars

We have cached all the simulated stars in the Twinkles field in the data folder of the repository. We have written methods to interact with the database.

In [2]:
truth_db_conn = desc.monitor.StarCacheDBObj(database='../data/star_cache.db')

In [3]:
truth_db_conn.columns

[('id', 'simobjid', int),
 ('raJ2000', 'ra*PI()/180.'),
 ('decJ2000', 'decl*PI()/180.'),
 ('glon', 'gal_l*PI()/180.'),
 ('glat', 'gal_b*PI()/180.'),
 ('properMotionRa', '(mura/(1000.*3600.))*PI()/180.'),
 ('properMotionDec', '(mudecl/(1000.*3600.))*PI()/180.'),
 ('parallax', 'parallax*PI()/648000000.'),
 ('galacticAv', 'CONVERT(float, ebv*3.1)'),
 ('radialVelocity', 'vrad'),
 ('variabilityParameters', 'varParamStr', str, 256),
 ('sedFilename', 'sedfilename', str, 256),
 ('simobjid', 'simobjid', int),
 ('ra', 'ra', float),
 ('decl', 'decl', float),
 ('magNorm', 'magNorm', float),
 ('mura', 'mura', float),
 ('mudecl', 'mudecl', float),
 ('ebv', 'ebv', float),
 ('vrad', 'vrad', float),
 ('varParamStr', 'varParamStr', str, 256),
 ('sedfilename', 'sedfilename', str, 256),
 ('gmag', 'gmag', float)]

## Use Opsim to calculate expected flux in visits.

We also have methods to interact with the LSST Opsim databases. This can be used to get the observing conditions simulated for each visit. We then use this information to create a pandas dataframe with the expected flux based upon the Opsim information and CatSim flux of the objects in the field.

In [4]:
worker = desc.monitor.TrueStars(truth_db_conn, '../../kraken_1042_sqlite.db')

In [5]:
# Just use one visit here for the demo. Otherwise it will use the list in data/selectedVisits.csv
star_df = worker.get_true_stars(for_obsHistIds=200, catalog_constraints='gmag > 11')

Generated 1 out of 1 obs_metadata
Generated fluxes for 1 out of 1 visits


/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sqlalchemy/1.0.8.lsst3+3/lib/python/SQLAlchemy-1.0.8-py2.7-linux-x86_64.egg/sqlalchemy/sql/elements.py:3779: SAWarning: Textual SQL expression 'ra between 52.671111 and ...' should be explicitly declared as text('ra between 52.671111 and ...') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})
/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sqlalchemy/1.0.8.lsst3+3/lib/python/SQLAlchemy-1.0.8-py2.7-linux-x86_64.egg/sqlalchemy/sql/elements.py:3779: SAWarning: Textual SQL expression 'gmag > 11' should be explicitly declared as text('gmag > 11') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})
/global/common/cori/contrib/lsst/lsstDM/twinkles_20170208/Linux64/sqlalchemy/1.0.8.lsst3+3/lib/python/SQLAlchemy-1.0.8-py2.7-linux-x86_64.egg/sqlalchemy/sql/sqltypes.py:565: SAWarning: Dialect sqlite+pysqlite does *not* supp

In [6]:
star_df

,uniqueId,ra,dec,filter,true_flux,true_flux_error,obsHistId
0,16285577220,52.903412,-27.189153,r,0.117658,0.040271,200
1,16288599044,52.964837,-27.207770,r,0.234963,0.040543,200
2,16291377156,52.779785,-27.259185,r,0.901140,0.042051,200
3,16295112708,53.016603,-27.257971,r,0.628290,0.041440,200
4,16311696388,52.957553,-27.413504,r,0.031243,0.040070,200
5,16314314756,52.691828,-27.473861,r,1.783972,0.043971,200
6,16332994564,53.160574,-27.570433,r,0.571313,0.041311,200
7,16342631428,53.017424,-27.670135,r,0.387217,0.040892,200
8,16346230788,52.930067,-27.712621,r,0.028436,0.040063,200
9,48113897476,53.128940,-27.288412,r,0.338467,0.040781,200


## Save to a sqlite database

Finally, we save this to a sqlite database to use in our comparisons to processed data from the simulated Twinkles images.

In [7]:
worker.write_to_db(star_df, 'twinkles_truth_demo.db', if_exists='replace')